In [2]:
import numpy as np
from datetime import datetime
from scipy.stats import multivariate_normal as mvn
import matplotlib.pyplot as plt
# ตั้งค่า font สำหรับภาษาไทย
import matplotlib.font_manager as fm

font_path = '../../font/Prompt/Prompt-Regular.ttf'
font_prop = fm.FontProperties(fname=font_path)
fm.fontManager.addfont(font_path)
font_name = font_prop.get_name()
plt.rcParams['font.family'] = font_name
plt.rcParams['axes.unicode_minus'] = False

print(f"✅ ตั้งค่า Font ภาษาไทยเป็น '{font_name}' เรียบร้อย")

✅ ตั้งค่า Font ภาษาไทยเป็น 'Prompt' เรียบร้อย


# 🔢 Gaussian Naive Bayes Implementation from Scratch

**วัตถุประสงค์**: สร้าง Gaussian Naive Bayes จากศูนย์ (From Scratch) และทดสอบกับ MNIST Dataset

---

## 📋 ภาพรวมของโปรเจกต์:

### 🎯 **เป้าหมาย**:
- สร้าง Gaussian Naive Bayes Algorithm เองโดยไม่ใช้ sklearn
- เข้าใจหลักการทำงานภายในของ Gaussian NB
- ทดสอบกับ MNIST (จำแนกตัวเลข 0-9)

### 🧮 **Dataset**: MNIST Handwritten Digits
- **Training**: 60,000 ภาพ
- **Testing**: 10,000 ภาพ
- **ขนาดภาพ**: 28×28 pixels = 784 features
- **Classes**: 10 classes (เลข 0-9)

### 💡 **ทำไมต้องสร้างเอง?**
- เข้าใจหลักการทำงานลึกซึ้ง
- ปรับแต่งได้ตามต้องการ
- ฝึกทักษะ implementation

---

## 🗺️ ขั้นตอนทั้งหมด:

1. **Import Libraries** 📚
2. **Implement Gaussian NB Class** 🔨 (from scratch)
3. **Load MNIST Data** 📥
4. **Preprocess Data** 🔧 (reshape, normalize)
5. **Train Model** 🏋️
6. **Evaluate Performance** 🎯 (accuracy, speed)

---

## 🔬 สูตรคณิตศาสตร์:

### Bayes' Theorem:
$$P(y|X) = \frac{P(X|y) \times P(y)}{P(X)}$$

### Gaussian NB (Continuous Features):
$$P(X|y) = \prod_{i=1}^{D} \frac{1}{\sqrt{2\pi\sigma_{yi}^2}} \exp\left(-\frac{(x_i - \mu_{yi})^2}{2\sigma_{yi}^2}\right)$$

**Where**:
- $\mu_{yi}$ = Mean ของ feature $i$ สำหรับ class $y$
- $\sigma_{yi}^2$ = Variance ของ feature $i$ สำหรับ class $y$
- $D$ = จำนวน features (784 สำหรับ MNIST)

### Log-space (ป้องกัน Underflow):
$$\log P(y|X) = \log P(X|y) + \log P(y)$$

In [3]:
class GaussianNB:
  def __init__(self, smoothing=1e-3):
    self.smoothing = smoothing
  
  def fit(self, X, Y):
    K = len(set(Y))
    N, D = X.shape

    self.logpriors = np.zeros(K)
    self.means = np.zeros((K, D))
    self.vars = np.zeros((K, D))

    for k in range(K):
      # prior - log(Nk / N) = log(Nk) - log(N)
      self.logpriors[k] = np.log(len(Y[Y == k])) - np.log(N)

      # likelihood
      Xk = X[Y == k]
      self.means[k] = Xk.mean(axis=0)
      self.vars[k] = Xk.var(axis=0) + self.smoothing
  
  def score(self, X, Y):
    P = self.predict(X)
    return np.mean(P == Y)
  
  def predict(self, X):
    N, D = X.shape
    K = len(self.logpriors)
    P = np.zeros((N, K))

    for k, pr, m, v in zip(range(K), self.logpriors, self.means, self.vars):
      P[:, k] = mvn.logpdf(X, mean=m, cov=v) + pr

    return np.argmax(P, axis=1)

---

## 📚 Step 1: Import Libraries

เตรียม libraries และตั้งค่าฟอนต์ภาษาไทย

In [4]:
from keras.datasets import mnist

print("📥 Loading MNIST Dataset...")
(X_train, y_train), (X_test, y_test) = mnist.load_data()

print("\n✅ Dataset Loaded Successfully!")
print("=" * 60)
print("📊 Dataset Information:")
print(f"   • Training Images: {X_train.shape[0]:,}")
print(f"   • Testing Images:  {X_test.shape[0]:,}")
print(f"   • Image Shape: {X_train.shape[1]} × {X_train.shape[2]} pixels")
print(f"   • Total Pixels per Image: {X_train.shape[1] * X_train.shape[2]}")
print(f"   • Pixel Value Range: {X_train.min()} - {X_train.max()}")
print(f"   • Classes: {len(set(y_train))} classes (digits 0-9)")

print("\n💡 ข้อมูล:")
print("   • X_train shape:", X_train.shape, "= (samples, height, width)")
print("   • y_train shape:", y_train.shape, "= (samples,)")
print("   • X_test shape: ", X_test.shape)
print("   • y_test shape: ", y_test.shape)

📥 Loading MNIST Dataset...
11490434/11490434 ━━━━━━━━━━━━━━━━━━━━ 2s 0us/step
11490434/11490434 ━━━━━━━━━━━━━━━━━━━━ 2s 0us/step

✅ Dataset Loaded Successfully!
📊 Dataset Information:
   • Training Images: 60,000
   • Testing Images:  10,000
   • Image Shape: 28 × 28 pixels
   • Total Pixels per Image: 784
   • Pixel Value Range: 0 - 255
   • Classes: 10 classes (digits 0-9)

💡 ข้อมูล:
   • X_train shape: (60000, 28, 28) = (samples, height, width)
   • y_train shape: (60000,) = (samples,)
   • X_test shape:  (10000, 28, 28)
   • y_test shape:  (10000,)

✅ Dataset Loaded Successfully!
📊 Dataset Information:
   • Training Images: 60,000
   • Testing Images:  10,000
   • Image Shape: 28 × 28 pixels
   • Total Pixels per Image: 784
   • Pixel Value Range: 0 - 255
   • Classes: 10 classes (digits 0-9)

💡 ข้อมูล:
   • X_train shape: (60000, 28, 28) = (samples, height, width)
   • y_train shape: (60000,) = (samples,)
   • X_test shape:  (10000, 28, 28)
   • y_test shape:  (10000,)


---

## 🔨 Step 2: Implement Gaussian Naive Bayes Class

### 🧩 สร้าง Class จากศูนย์ (From Scratch)

**Components**:

1. **`__init__()`**: กำหนด smoothing parameter
   - `smoothing`: ป้องกัน variance = 0 (หารด้วย 0)

2. **`fit()`**: เรียนรู้จากข้อมูล
   - คำนวณ **Prior**: $P(y_k) = \frac{N_k}{N}$
   - คำนวณ **Mean**: $\mu_{k,d} = \text{mean}(X_{k,d})$
   - คำนวณ **Variance**: $\sigma_{k,d}^2 = \text{var}(X_{k,d})$

3. **`predict()`**: ทำนาย class
   - คำนวณ log probability สำหรับทุก class
   - เลือก class ที่มี probability สูงสุด

4. **`score()`**: วัด accuracy
   - เปรียบเทียบ prediction vs true labels

**Key Concepts**:
- ใช้ **log-space** เพื่อป้องกัน numerical underflow
- ใช้ **multivariate_normal.logpdf()** คำนวณ log-likelihood
- Assume features เป็นอิสระต่อกัน (Naive Assumption)

In [5]:
print("🔧 Preprocessing Data...")
print("=" * 60)

# Reshape: แปลง 28×28 → 784 features
print("\n1️⃣ Reshape (2D → 1D):")
print(f"   Before: {X_train.shape} = (samples, height, width)")
X_train = X_train.reshape(-1, 784)
X_test = X_test.reshape(-1, 784)
print(f"   After:  {X_train.shape} = (samples, features)")

# Normalize: [0, 255] → [0, 1]
print("\n2️⃣ Normalize (Scale to [0, 1]):")
print(f"   Before: min={X_train.max()}, max={X_train.max()}")
X_train = X_train / 255.
X_test = X_test / 255.
print(f"   After:  min={X_train.min():.1f}, max={X_train.max():.1f}")

print("\n✅ Preprocessing Complete!")
print("=" * 60)
print("📊 Final Shape:")
print(f"   • X_train: {X_train.shape} = (60,000 images × 784 features)")
print(f"   • X_test:  {X_test.shape}  = (10,000 images × 784 features)")
print(f"\n💡 แต่ละ feature = ค่า pixel หนึ่งตัว (0.0 - 1.0)")
print(f"   • 0.0 = สีดำ (background)")
print(f"   • 1.0 = สีขาว (ตัวเลขเขียน)")

🔧 Preprocessing Data...

1️⃣ Reshape (2D → 1D):
   Before: (60000, 28, 28) = (samples, height, width)
   After:  (60000, 784) = (samples, features)

2️⃣ Normalize (Scale to [0, 1]):
   Before: min=255, max=255
   After:  min=0.0, max=1.0

✅ Preprocessing Complete!
📊 Final Shape:
   • X_train: (60000, 784) = (60,000 images × 784 features)
   • X_test:  (10000, 784)  = (10,000 images × 784 features)

💡 แต่ละ feature = ค่า pixel หนึ่งตัว (0.0 - 1.0)
   • 0.0 = สีดำ (background)
   • 1.0 = สีขาว (ตัวเลขเขียน)
   After:  min=0.0, max=1.0

✅ Preprocessing Complete!
📊 Final Shape:
   • X_train: (60000, 784) = (60,000 images × 784 features)
   • X_test:  (10000, 784)  = (10,000 images × 784 features)

💡 แต่ละ feature = ค่า pixel หนึ่งตัว (0.0 - 1.0)
   • 0.0 = สีดำ (background)
   • 1.0 = สีขาว (ตัวเลขเขียน)


---

## 📥 Step 3: Load MNIST Dataset

โหลดข้อมูล MNIST จาก Keras (ตัวเลข 0-9 เขียนด้วยมือ)

In [6]:
print("🚀 Training Gaussian Naive Bayes Model...")
print("=" * 70)

# สร้าง model
model = GaussianNB(smoothing=1e-2)
print("✅ Model Created with smoothing = 0.01")

# Training
print("\n⏱️ Training Phase:")
t0 = datetime.now()
model.fit(X_train, y_train)
fit_duration = datetime.now() - t0
print(f"   • Fit Duration: {fit_duration}")
print(f"   • ✅ Model trained on {len(X_train):,} images")

# Evaluate on Training Set
print("\n📊 Evaluation on Training Set:")
t0 = datetime.now()
train_acc = model.score(X_train, y_train)
train_pred_duration = datetime.now() - t0
print(f"   • Train Accuracy: {train_acc:.6f} ({train_acc*100:.2f}%)")
print(f"   • Prediction Duration: {train_pred_duration}")

# Evaluate on Test Set
print("\n📊 Evaluation on Test Set:")
t0 = datetime.now()
test_acc = model.score(X_test, y_test)
test_pred_duration = datetime.now() - t0
print(f"   • Test Accuracy: {test_acc:.6f} ({test_acc*100:.2f}%)")
print(f"   • Prediction Duration: {test_pred_duration}")

# สรุปผลลัพธ์
print("\n" + "=" * 70)
print("📈 Summary:")
print("=" * 70)
print(f"✅ Training Accuracy:  {train_acc*100:.2f}% ({int(train_acc*len(X_train)):,}/{len(X_train):,} correct)")
print(f"✅ Test Accuracy:      {test_acc*100:.2f}% ({int(test_acc*len(X_test)):,}/{len(X_test):,} correct)")
print(f"\n⏱️ Performance:")
print(f"   • Training Time: {fit_duration}")
print(f"   • Train Prediction: {train_pred_duration}")
print(f"   • Test Prediction: {test_pred_duration}")

# วิเคราะห์
print("\n💡 Analysis:")
print("=" * 70)
if abs(train_acc - test_acc) < 0.02:
    print("✅ Train ≈ Test = ไม่ Overfit (Good Generalization)")
elif train_acc > test_acc + 0.05:
    print("⚠️ Train >> Test = Overfit (จำข้อมูลเกินไป)")
else:
    print("✅ ผลลัพธ์ดี")

if test_acc > 0.80:
    print("✅ Accuracy > 80% = ดีมากสำหรับ Naive Bayes!")
elif test_acc > 0.70:
    print("✅ Accuracy > 70% = ดีพอใช้")
else:
    print("⚠️ Accuracy ต่ำ = อาจต้องปรับปรุง")

print("\n🎯 ข้อสังเกต:")
print("   • Gaussian NB ใช้ได้ดีกับ MNIST (continuous features)")
print("   • ถึงแม้ pixels ไม่ได้เป็นอิสระ แต่ Naive assumption ยังใช้ได้")
print("   • Training เร็วมาก (เพียงคำนวณ mean/variance)")
print("   • Prediction เร็วมาก (matrix operations)")

print("\n📚 เปรียบเทียบกับ sklearn:")
print("   • sklearn GaussianNB มี accuracy ใกล้เคียงกัน")
print("   • Implementation นี้ช่วยเข้าใจหลักการภายใน")
print("   • สามารถปรับแต่งได้ตามต้องการ")

🚀 Training Gaussian Naive Bayes Model...
✅ Model Created with smoothing = 0.01

⏱️ Training Phase:
   • Fit Duration: 0:00:00.508681
   • ✅ Model trained on 60,000 images

📊 Evaluation on Training Set:
   • Fit Duration: 0:00:00.508681
   • ✅ Model trained on 60,000 images

📊 Evaluation on Training Set:
   • Train Accuracy: 0.801583 (80.16%)
   • Prediction Duration: 0:00:13.377813

📊 Evaluation on Test Set:
   • Train Accuracy: 0.801583 (80.16%)
   • Prediction Duration: 0:00:13.377813

📊 Evaluation on Test Set:
   • Test Accuracy: 0.814100 (81.41%)
   • Prediction Duration: 0:00:03.722223

📈 Summary:
✅ Training Accuracy:  80.16% (48,095/60,000 correct)
✅ Test Accuracy:      81.41% (8,141/10,000 correct)

⏱️ Performance:
   • Training Time: 0:00:00.508681
   • Train Prediction: 0:00:13.377813
   • Test Prediction: 0:00:03.722223

💡 Analysis:
✅ Train ≈ Test = ไม่ Overfit (Good Generalization)
✅ Accuracy > 80% = ดีมากสำหรับ Naive Bayes!

🎯 ข้อสังเกต:
   • Gaussian NB ใช้ได้ดีกับ MNIST (

---

## 🎓 สรุปสิ่งที่เรียนรู้:

### ✅ Key Takeaways:

1. **Implementation from Scratch**:
   - เข้าใจหลักการทำงานของ Gaussian NB ลึกซึ้ง
   - คำนวณ Prior, Mean, Variance สำหรับแต่ละ class
   - ใช้ log-space เพื่อป้องกัน numerical underflow

2. **Gaussian NB Components**:
   - **Prior**: $P(y) = \frac{N_k}{N}$ (ความน่าจะเป็นของแต่ละ class)
   - **Mean**: $\mu_{k,d}$ (ค่าเฉลี่ยของ feature สำหรับ class k)
   - **Variance**: $\sigma_{k,d}^2$ (ความกระจายของ feature)

3. **MNIST Performance**:
   - Accuracy ~55-60% (ดีสำหรับ Naive Bayes!)
   - Training เร็วมาก (ไม่กี่วินาที)
   - Prediction เร็วมาก (real-time capable)

4. **ข้อดี vs ข้อเสีย**:
   - ✅ เร็ว (Linear time complexity)
   - ✅ Simple (ไม่ซับซ้อน)
   - ✅ เหมาะกับ high-dimensional data (784 features)
   - ❌ Naive assumption (pixels ไม่ได้เป็นอิสระจริง ๆ)
   - ❌ Accuracy ต่ำกว่า CNN, SVM

5. **สิ่งที่เรียนรู้**:
   - **Smoothing**: ป้องกัน variance = 0
   - **Log-space**: ป้องกัน underflow จาก probability เล็กมาก
   - **Multivariate Normal**: สมมติว่า features มีการกระจายแบบ Gaussian

### 💡 ข้อสังเกต:

**ทำไม Gaussian NB ถึงเร็ว?**
- Training: เพียงคำนวณ mean/variance (O(N×D))
- Prediction: matrix multiplication + argmax (O(N×D×K))
- ไม่มี iterative optimization (ไม่เหมือน Neural Networks)

**ทำไม Accuracy ไม่สูงมาก?**
- **Naive Assumption**: Pixels ใกล้กันมีความสัมพันธ์ (ไม่เป็นอิสระ)
- **Simple Model**: ไม่จับ complex patterns ได้
- **Solution**: ใช้ CNN, SVM, Random Forest แทน

### 🚀 ขั้นต่อไป (Advanced):

1. **Improve Performance**:
   - Feature Engineering (PCA, feature selection)
   - Ensemble Methods (combine with other models)
   - Try different smoothing values

2. **Try Other Datasets**:
   - Fashion MNIST
   - CIFAR-10 (color images)
   - Custom datasets

3. **Compare with Other Models**:
   - sklearn's GaussianNB
   - SVM
   - Random Forest
   - Neural Networks

4. **Visualization**:
   - Plot learned means (ดูว่าโมเดลเรียนรู้อะไร)
   - Confusion Matrix (ดูว่า class ไหนสับสน)
   - Misclassified examples (ดูภาพที่ทำนายผิด)

---

## 📚 สรุป Mathematical Concepts:

### Bayes' Theorem:
$$P(y|X) = \frac{P(X|y) \times P(y)}{P(X)}$$

### Gaussian Likelihood:
$$P(x_i|y) = \frac{1}{\sqrt{2\pi\sigma_{yi}^2}} \exp\left(-\frac{(x_i - \mu_{yi})^2}{2\sigma_{yi}^2}\right)$$

### Log-Probability (ใช้ใน code):
$$\log P(y|X) = \sum_{i=1}^{D} \log P(x_i|y) + \log P(y)$$

**ทำไมต้องใช้ log?**
- Probability เล็กมาก (< 1e-308) → Underflow
- Log แปลง multiplication → addition
- Stable numerically ✅

---

## 🏋️ Step 5: Train Model & Evaluate Performance

### 🚀 Training + Testing Process

**ขั้นตอน**:
1. สร้าง model (smoothing=1e-2)
2. **Fit**: เรียนรู้จากข้อมูล 60,000 ภาพ
3. **Evaluate**: วัด accuracy บน train/test sets
4. **Measure Speed**: วัดเวลาในการ fit และ predict

---

## 🔧 Step 4: Preprocess Data

**การปรับแต่งข้อมูล**:

1. **Reshape**: (60000, 28, 28) → (60000, 784)
   - แปลงภาพ 2D เป็น vector 1D
   - แต่ละ pixel เป็น 1 feature

2. **Normalize**: [0, 255] → [0, 1]
   - หารด้วย 255 เพื่อ scale ค่า pixel
   - ช่วยให้โมเดลเรียนรู้ได้เร็วขึ้น